# Time Series Prediction using simple RNN and LSTM

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Improrting libraries
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import yfinance as yf
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, SimpleRNN
import TS_LSTM as tspy

keras.backend.set_image_data_format("channels_last")
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight") 
%matplotlib inline

In [ ]:
import importlib
importlib.reload(tspy)

## Data Preprocessing and Visualization

In [ ]:
dataset = yf.download('NVDA', start='2025-03-24', end='2025-03-29', interval='1m')
dataset = dataset.between_time('09:30', '16:00')

dataset.head(200)

In [ ]:
tstart = datetime.datetime(2025, 3, 24)
tend = datetime.datetime(2025, 3, 29)

tspy.train_test_plot(dataset, tstart, tend)

In [ ]:
training_set, test_set = tspy.train_test_split(dataset, tstart, tend)
sc = MinMaxScaler(feature_range=(0, 1))
training_set = training_set.reshape(-1, 1)
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
n_steps = 1      # Number of time steps in each sequence
features = 1     # Number of features in each time step

X_train, y_train = tspy.split_sequence(training_set_scaled, n_steps)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], features)

In [ ]:
inputs = sc.transform(test_set.reshape(-1, 1))

X_test, y_test = tspy.split_sequence(inputs, n_steps)
X_test = X_test.reshape(-1, n_steps, features)

## Time Series Prediction using RNN

In [ ]:
model_rnn = Sequential()
model_rnn.add(SimpleRNN(units=125, input_shape=(n_steps, features)))
model_rnn.add(Dense(units=1))
model_rnn.compile(optimizer="RMSprop", loss="mse")

history = model_rnn.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

In [ ]:
predicted_stock_price = model_rnn.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
tspy.plot_predictions(test_set, predicted_stock_price, "IBIT Stock Price")

## Time Series Prediction using LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(units=125, return_sequences=True, input_shape=(n_steps, features)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=50, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=50, return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(units=1))
model_lstm.compile(optimizer="RMSprop", loss="mse")

model_lstm.summary()

In [ ]:
history = model_lstm.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

predicted_stock_price = model_lstm.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
tspy.plot_predictions(test_set, predicted_stock_price, "NVDA Stock Price")

In [ ]:
# After training your model
model_lstm.save('model_lstm.keras')

# And saving your scaler (assuming you're using scikit-learn's scaler)
import pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(sc, f)
